In [17]:
def parse_trace_line(line):
    """Extract relevant fields from a trace line."""
    parts = line.split()
    event = parts[0]  # +, -, r, or d
    time = float(parts[1])
    src = int(parts[2])
    dst = int(parts[3])
    pkt_size = int(parts[5]) * 8  # packet size in bits
    flow_id = int(parts[7])  
    return event, time, src, dst, pkt_size, flow_id


def calculate_throughput_and_loss(trace_file, src_node, dst_node, start_time, flow_id):
    """Calculate average throughput and packet loss rate for a specific flow starting from start_time."""
    bits_received = 0
    first_packet_time = None
    last_packet_time = None
    received_packets = 0
    dropped_packets = 0

    with open(trace_file, 'r') as file:
        for line in file:
            parsed = parse_trace_line(line)
            if parsed is None:
                continue
            event, time, src, dst, pkt_size, fid = parsed

            # Process only events for the specified flow_id
            if fid == flow_id and src == src_node and dst == dst_node and time >= start_time:
                if event == 'r':  # Received packets
                    if first_packet_time is None:
                        first_packet_time = time

                    # Update total bits and track time for throughput calculation
                    bits_received += pkt_size
                    last_packet_time = time
                    received_packets += 1

                elif event == 'd':  # Dropped packets
                    dropped_packets += 1

    if first_packet_time and last_packet_time and last_packet_time > first_packet_time:
        # Calculate throughput (bits per second)
        duration = last_packet_time - first_packet_time
        throughput = bits_received / duration  # bits per second
        total_packets = received_packets + dropped_packets
        loss_rate = dropped_packets / total_packets  # loss rate as a fraction
    else:
        throughput = 0.0  # No packets received in the time frame
        loss_rate = 0.0  # No packets to compute loss rate

    return throughput, loss_rate

# Example usage
trace_file = 'out.tr'  # Replace with the path to your trace file
src_node = 2  # Source node
dst_node = 3  # Destination node
start_time = 1.0  # Start time for throughput calculation
flow_id = 1  # Set flow_id to a specific integer

# Compute throughput, received packets, dropped packets, and loss rate
tput, loss_rate = calculate_throughput_and_loss(trace_file, src_node, dst_node, start_time, flow_id)

# Display results
print(f"Average throughput from node {src_node} to node {dst_node} after time {start_time} for flow {flow_id} is {tput:.1f} bps, loss rate is {loss_rate:.4f}")


Average throughput from node 2 to node 3 after time 1.0 for flow 1 is 284149.5 bps, loss rate is 0.0119
